In [1]:
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments
import pandas as pd
import evaluate
import numpy as np

In [3]:
df = pd.read_parquet("数据/多项选择模型.parquet")
print(df.shape)
df.head()

(100, 4)


,documents,question,choice,answer
0,"[男：你今天晚上有时间吗?我们一起去看电影吧?, 女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片...",女的最喜欢哪种电影?,"[恐怖片, 爱情片, 喜剧片, 科幻片]",喜剧片
1,"[男：足球比赛是明天上午八点开始吧?, 女：因为天气不好，比赛改到后天下午三点了。]",根据对话，可以知道什么?,"[今天天气不好, 比赛时间变了, 校长忘了时间]",比赛时间变了
2,"[女：今天下午的讨论会开得怎么样?, 男：我觉得发言的人太少了。]",关于这次讨论会，我们可以知道什么?,"[会是昨天开的, 男的没有参加, 讨论得不热烈, 参加的人很少]",讨论得不热烈
3,"[男：我记得你以前很爱吃巧克力，最近怎么不吃了，是在减肥吗?, 女：是啊，我希望自己能瘦一点儿。]",女的为什么不吃巧克力了?,"[刷牙了, 要减肥, 口渴了, 吃饱了]",要减肥
4,"[女：过几天刘明就要从英国回来了。我还真有点儿想他了，记得那年他是刚过完中秋节走的。, 男：...",现在大概是哪一年?,"[2005年, 2010年, 2008年, 2009年]",2010年


In [4]:
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['documents', 'question', 'choice', 'answer'],
        num_rows: 90
    })
    test: Dataset({
        features: ['documents', 'question', 'choice', 'answer'],
        num_rows: 10
    })
})

## 2、数据预处理

In [5]:
model_path = "hfl/chinese-macbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)

C:\Users\jtyou\miniconda3\envs\torch\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def process_func(examples):
    contexts, questions, labels = [], [], []

    documents = examples["documents"]
    for index, document in enumerate(documents):
        context = "\n".join(document)
        question = examples["question"][index]
        choices = examples["choice"][index] + ["不知道"] * 4
        answer = examples["answer"][index]

        for i in range(4):
            contexts.append(context)
            questions.append(f"{question}{choices[i]}")

        labels.append(choices.index(answer))

    token = tokenizer(text=contexts,
                      text_pair=questions,
                      max_length=128,
                      truncation="only_first",
                      padding="max_length",
                      return_tensors="pt",
                      )

    token = {k: v.reshape(-1, 4, 128) for k, v in token.items()}
    token["labels"] = labels
    return token

In [8]:
dataloader = dataset.map(process_func, batched=True)
dataloader["test"]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['documents', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

## 3、构建模型

In [9]:
model = AutoModelForMultipleChoice.from_pretrained(model_path)

C:\Users\jtyou\miniconda3\envs\torch\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\jtyou\miniconda3\envs\torch\Lib\site-packages\transformers\modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted b

## 4、创建评估函数

In [10]:
accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    prediction, label = pred
    p = np.argmax(prediction, axis=-1)
    return accuracy.compute(predictions=p, references=label)

Using the latest cached version of the module from C:\Users\jtyou\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Fri Jan  3 23:07:59 2025) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


## 5、创建训练参数

In [12]:
args = TrainingArguments(
    output_dir="模型",
    eval_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=1e-5,
    num_train_epochs=3,
    eval_steps=100,
    logging_steps=100,
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
)

## 6、创建训练器

In [13]:
train = Trainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
    train_dataset=dataloader["train"],
    eval_dataset=dataloader["test"]
)

## 7、训练

In [14]:
train.train()

The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: documents, choice, question, answer. If documents, choice, question, answer are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
C:\Users\jtyou\miniconda3\envs\torch\Lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 90
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 36
  Number of trainable parameters = 102268417


Step,Training Loss,Validation Loss



KeyboardInterrupt

